In [ ]:
import os
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image

# Step 1: Download and Prepare the Dataset
# Organize the dataset into 'cats' and 'dogs' folders.

# Step 2: Data Preprocessing
# Load and preprocess images

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        if img_path.endswith(".jpg"):
            img = preprocess_image(img_path)
            images.append(img)
    return np.vstack(images)

cats_features = load_images_from_folder('cats')
dogs_features = load_images_from_folder('dogs')
labels = np.concatenate((np.zeros(len(cats_features)), np.ones(len(dogs_features))))

# Step 3: Feature Extraction (using VGG16)
base_model = VGG16(weights='imagenet', include_top=False)
cats_features = base_model.predict(cats_features)
dogs_features = base_model.predict(dogs_features)

# Step 4: Train the SVM
X = np.vstack((cats_features, dogs_features))
X = X.reshape(X.shape[0], -1)
X_train, X_val, y_train, y_val = train_test_split(X, labels, test_size=0.2, random_state=42)

svm_classifier = SVC(C=1.0, kernel='linear')
svm_classifier.fit(X_train, y_train)

# Step 6: Evaluate the Model
y_pred = svm_classifier.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy: {accuracy}")

# Additional evaluation metrics (precision, recall, F1-score)
print(classification_report(y_val, y_pred))

# Step 7: Testing
# Load and preprocess the test dataset and use svm_classifier.predict() to make predictions.